In [10]:
using JuMP
using Gurobi
using Random

In [11]:
N=6 #No. of nodes (1 supplier + 5 DC's)

K=3 #No. of vehicles

Dk=900 #Capacity of vehicles

T=8 #Periods

h=0.025 #Holding cost/unit at each DC

C=[5000 502 488 220 486 742] #Capacity limit for (1 supplier + 5 DC's)

d=[0 0 0 0 0 0 0 0;
75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9;
62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9;
67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2;
102.3 156 156 156 156 119.05 65.35 102.3;
107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
] #Demand for each day and (1 supplier + 5 DC's)

c=[
0 140 434 389 419 125;
140 0 300 455 400 97;
434 300 0 609 417 330;
389 455 609 0 256 358;
419 400 417 256 0 316;
125 97 330 358 316 0
] #Transportation cost

std=[
0 0 0 0 0 0 0 0;
11.87 11.87 11.87 11.87 11.87 14.69 14.69 11.87;
10.91 13.89 13.89 13.89 13.89 14.14 11.22 10.91;
11.27 11.27 11.27 11.27 11.27 12.86 12.86 11.27;
13.19 16.03 16.03 16.03 16.03 15.05 11.98 13.19;
13.04 14.00 14.00 14.00 14.00 11.08 9.84 13.04
]# Standard deviation of Demand

I0=[0 98.8 153.9 42.4 23.4 85.7] #Initial inventory


1×6 Matrix{Float64}:
 0.0  98.8  153.9  42.4  23.4  85.7

In [12]:
model=Model(Gurobi.Optimizer)
@variable(model, x[i=1:N,j=1:N,k=1:K,t=1:T], Bin); #If truck k leaves DC i and goes to DC j
@variable(model, y[i=1:N,k=1:K,t=1:T], Bin); #If truck k arrives at DC i
@variable(model, z[j=1:N,k=1:K,t=1:T] >= 0); #Load of truck k when arriving at node j
@variable(model, I[i=1:N,t=0:T] >= 0); #Inventory level at node i till end of period t
@variable(model, q[i=1:N,k=1:K,t=1:T] >= 0); #Quantity delivered to customer i by truck k at start of period t

@objective(model, Min, sum(h*I[i,t] for i=2:N, t=1:T) + sum(c[i,j]*x[i,j,k,t]  for i=1:N, j=1:N, k=1:K, t=1:T))

@constraint(model, [i=2:N], I[i,0] == I0[i]) #Initializing inventory level

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) == I[i,t] + d[i,t]); #Balance constraint for each DC

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) <= C[i]); #Capacity constraint for each DC

@constraint(model, [i=2:N,k=1:K,t=1:T], q[i,k,t] <= C[i]*y[i,k,t]); #Constraint which ensures y is 1 when they receive a quantity

@constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i=2:N) <= Dk*y[1,k,t]); #If truck delivers from supplier, it has to activate the supplier

@constraint(model, [h=1:N,k=1:K,t=1:T], sum(i == h ? 0 : x[i,h,k,t] for i = 1:N) == y[h,k,t]); #If truck visits DC

@constraint(model, [h=1:N,k=1:K,t=1:T], sum(j == h ? 0 : x[h,j,k,t] for j = 1:N) == y[h,k,t]); #If truck leaves DC

@constraint(model, [i=2:N,j=2:N,k=1:K,t=1:T], z[i,k,t] - d[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d)); #To avoid subtours automatically



Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-06


In [13]:
optimize!(model)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0xded68cbd
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+03]
Presolve removed 160 rows and 213 columns
Presolve time: 0.02s
Presolved: 957 rows, 1137 columns, 3764 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 2.649403e+03, 668 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2649.40295    0   92          - 2649.40295      -     -    0s
H    0     0                    10097.895000 2649.40295  73.8%     -   

In [14]:
if termination_status(model) == MOI.OPTIMAL
   println("RESULTS:")
   println("Objective = $(objective_value(model))")
else
   println("No solution")
end

println();
    for k=1:K
        has_route = false;
        print("Truck:$(value(k))\n")
        for t=1:T
            print("Day:$(value(t))\n")
            for i=1:N
                for j=1:N
                    if (value(x[i,j,k,t]) == 1)
                        print(j,"->",i,"\n")
                        print("$(value(q[i,k,t]))\n")
                        has_route = true;
                    end
                end
            end
        end
        if has_route
            println();
        end
    end

for i=1:N
    for k=1:K
        for t=1:T
            if (value(q[i,k,t]) > 0)
                println(" Node ",i," Truck ",k," Day ", t," Delivery ",value(q[i,k,t]))
            end
        end
    end
end


for t=1:T
    println("Day", t,)
    for i=2:N
        println("Node ",i," Inventory Level = ",value(I[i,t]))
    end
end


RESULTS:
Objective = 4481.569999999999

Truck:1
Day:1
6->1
0.0
1->6
526.299999999992
Day:2
Day:3
6->1
0.0
1->4
201.60000000000005
4->5
467.5999999999929
5->6
230.80000000000706
Day:4
Day:5
Day:6
Day:7
Day:8

Truck:2
Day:1
2->1
0.0
3->2
280.6999999999992
5->3
291.99999999999454
1->4
91.99999999999972
4->5
235.30000000000638
Day:2
Day:3
Day:4
Day:5
Day:6
4->1
0.0
1->2
247.29999999999865
2->3
164.50000000000347
5->4
201.49999999999952
3->5
286.6999999999984
Day:7
Day:8

Truck:3
Day:1
Day:2
Day:3
Day:4
Day:5
Day:6
Day:7
Day:8
 Node 2 Truck 2 Day 1 Delivery 280.6999999999992
 Node 2 Truck 2 Day 6 Delivery 247.29999999999865
 Node 3 Truck 2 Day 1 Delivery 291.99999999999454
 Node 3 Truck 2 Day 6 Delivery 164.50000000000347
 Node 4 Truck 1 Day 3 Delivery 201.60000000000005
 Node 4 Truck 2 Day 1 Delivery 91.99999999999972
 Node 4 Truck 2 Day 6 Delivery 201.49999999999952
 Node 5 Truck 1 Day 3 Delivery 467.5999999999929
 Node 5 Truck 2 Day 1 Delivery 235.30000000000638
 Node 5 Truck 2 Day 6 Del

In [15]:
#Random.seed!(0);
#counter=zeros(6,8)
#probability=zeros(6,8)
#for r = 1:10000
#    actual_demand = d + randn(6,8).*std;
#    for i=1:N
#        new_IL=I0[i];
#        for t=1:T
#            delivered=0;
#            for k=1:K
#                delivered = delivered + value(q[i,k,t]);
#            end
#            new_IL=new_IL+delivered-value(actual_demand[i,t]);
#            fillrate=(max(value(actual_demand[i,t])+min(new_IL,0),0))/value(actual_demand[i,t]);
#            if fillrate>0.95
#                counter[i,t]=counter[i,t] + 1
#                probability[i,t]=counter[i,t]/10000
#            end
#        end
#    end
#end

In [16]:
#probability # Probability of meeting the fillrate target

In [19]:
#actual_demand = d + randn(6,8).*std;
Random.seed!(0);
counter=zeros(5,8)

dd=[75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9;
62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9;
67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2;
102.3 156 156 156 156 119.05 65.35 102.3;
107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4]

stdd=[
11.87 11.87 11.87 11.87 11.87 14.69 14.69 11.87;
10.91 13.89 13.89 13.89 13.89 14.14 11.22 10.91;
11.27 11.27 11.27 11.27 11.27 12.86 12.86 11.27;
13.19 16.03 16.03 16.03 16.03 15.05 11.98 13.19;
13.04 14.00 14.00 14.00 14.00 11.08 9.84 13.04
]# Sta

for r = 1:10000
    actual_demand = dd + randn(5,8).*stdd;
    for i=1:N-1
        new_IL=I0[i+1];
        for t=1:T
            #fillrate=0;
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i+1,k,t]);
            end
            new_IL=new_IL+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(new_IL,0),0))/value(actual_demand[i,t]);
            if fillrate>0.95
                counter[i,t] = counter[i,t] + 1
            end
        end
    end
end
=counter/10000

5×8 Matrix{Float64}:
 1.0  1.0     1.0  0.9997  0.5613  1.0     0.9923  0.5434
 1.0  1.0     1.0  0.9999  0.6151  0.9988  0.9683  0.5354
 1.0  0.5799  1.0  0.9995  0.5545  1.0     0.9885  0.5421
 1.0  0.657   1.0  1.0     0.5939  1.0     0.9953  0.5532
 1.0  1.0     1.0  1.0     1.0     1.0     0.9995  0.5575

In [18]:
probability# Probability of meeting the fillrate target

5×8 Matrix{Float64}:
 0.1  0.1      0.1  0.09997  0.05613  0.1      0.09923  0.05434
 0.1  0.1      0.1  0.09999  0.06151  0.09988  0.09683  0.05354
 0.1  0.05799  0.1  0.09995  0.05545  0.1      0.09885  0.05421
 0.1  0.0657   0.1  0.1      0.05939  0.1      0.09953  0.05532
 0.1  0.1      0.1  0.1      0.1      0.1      0.09995  0.05575